В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


In [3]:
def proportions_diff_z_stat_rel(f, g, n):
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [4]:
proportions_diff_z_stat_rel(24, 4, 50)

4.47213595499958

In [5]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [9]:
proportions_diff_z_test(proportions_diff_z_stat_rel(24, 4, 50), 'greater')

3.8721082155079856e-06

In [10]:
np.round(proportions_diff_z_test(proportions_diff_z_stat_rel(24, 4, 50), 'greater'), 4)

0.0

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

In [13]:
diam = pd.read_table('banknotes.txt')
diam.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [14]:
y_diam = diam['real']
diam.drop('real', axis=1, inplace=True)

In [15]:
X_train, X_test, y_train, y_test  = train_test_split(diam, y_diam, train_size=150, random_state=1)

X_train.shape

(150, 6)

Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

логистическая регрессия по признакам X_1,X_2,X_3X 

логистическая регрессия по признакам X_4,X_5,X_6X 

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5\times10^{-8}5.5×10 
−8
 , нужно ввести 8).

In [30]:
reg1 = LogisticRegression().fit(X_train[['X1', 'X2', 'X3']], y_train)
reg2 = LogisticRegression().fit(X_train[['X4', 'X5', 'X6']], y_train)

In [31]:
test = X_test.copy()
test['reg1'] = reg1.predict(X_test[['X1', 'X2', 'X3']])
test['reg2'] = reg2.predict(X_test[['X4', 'X5', 'X6']])
test.head()

,X1,X2,X3,X4,X5,X6,reg1,reg2
58,215.1,130.1,129.9,7.9,11.0,141.3,1,1
40,214.4,129.8,129.2,8.9,9.4,142.3,1,1
34,215.9,130.4,130.0,8.9,10.6,141.4,1,1
102,214.9,130.3,130.1,8.7,11.7,140.2,0,0
184,214.7,130.2,130.1,10.7,11.1,139.5,0,0


In [32]:
test['real'] = y_test

In [33]:
def mist(x):
    if x[1] != x[0]:
        return 1
    return 0

In [34]:
test['reg1_mist'] = test[['reg1', 'real']].apply(mist, axis=1)
test['reg2_mist'] = test[['reg2', 'real']].apply(mist, axis=1)

In [35]:
test.head()

,X1,X2,X3,X4,X5,X6,reg1,reg2,real,reg1_mist,reg2_mist
58,215.1,130.1,129.9,7.9,11.0,141.3,1,1,1,0,0
40,214.4,129.8,129.2,8.9,9.4,142.3,1,1,1,0,0
34,215.9,130.4,130.0,8.9,10.6,141.4,1,1,1,0,0
102,214.9,130.3,130.1,8.7,11.7,140.2,0,0,0,0,0
184,214.7,130.2,130.1,10.7,11.1,139.5,0,0,0,0,0


In [37]:
def proportions_diff_z_stat_rel2(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [41]:
z = proportions_diff_z_stat_rel2(test.reg1_mist, test.reg2_mist)

In [42]:
proportions_diff_z_test(z)

0.0032969384555543435

В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.



In [51]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [53]:
np.round(proportions_diff_confint_rel(test.reg1_mist, test.reg2_mist), 4)

array([0.0599, 0.3001])

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [43]:
import math

In [61]:
zstat = (525-541.4) *np.sqrt(100.) / 100.
print zstat

-1.6399999999999977


In [62]:
np.round(1 - stats.norm.cdf(abs(zstat)), 4)

0.0505

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.